# YUV Processing Pipeline (Notebook Version)

This notebook provides a step-by-step pipeline for:
1. Decoding `.264` pristine videos to `.yuv` in multiple resolutions.
2. Encoding `.yuv` files with different CRF levels into `.264` (H.264 codec).
3. Decoding encoded `.264` files back to `.yuv` for evaluation.


In [ ]:
import os
import subprocess
from pathlib import Path

def run_command(cmd):
    print("Running:", " ".join(cmd))
    subprocess.run(cmd, check=True)


In [ ]:
def decode_pristine_to_yuv(input_dir, output_dir, resolutions):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    for f264 in input_dir.glob("*.264"):
        for res in resolutions:
            out_res_dir = output_dir / res
            out_res_dir.mkdir(parents=True, exist_ok=True)
            name = f264.stem
            out_path = out_res_dir / f"{name}.yuv"
            width, height = map(int, res.split('x'))
            cmd = [
                "ffmpeg", "-y", "-s", "1920x1080", "-i", str(f264),
                "-vf", f"scale={width}:{height}:flags=lanczos",
                "-pix_fmt", "yuv420p", str(out_path)
            ]
            run_command(cmd)


In [ ]:
def encode_yuv_to_h264(input_dir, output_dir, resolutions, crf_list):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)

    for res in resolutions:
        res_dir = input_dir / res
        for yuv_file in res_dir.glob("*.yuv"):
            for crf in crf_list:
                out_dir = output_dir / res / str(crf)
                out_dir.mkdir(parents=True, exist_ok=True)
                out_file = out_dir / f"{yuv_file.stem}.264"
                width, height = map(int, res.split('x'))
                cmd = [
                    "ffmpeg", "-y", "-s", f"{width}x{height}", "-pix_fmt", "yuv420p", "-i", str(yuv_file),
                    "-c:v", "libx264", "-preset", "slow", "-crf", str(crf), "-f", "h264", str(out_file)
                ]
                run_command(cmd)


In [ ]:
def decode_encoded_to_yuv(input_dir, output_dir, resolutions, crf_list):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)

    for res in resolutions:
        for crf in crf_list:
            crf_dir = input_dir / res / str(crf)
            for f264 in crf_dir.glob("*.264"):
                out_dir = output_dir / res / str(crf)
                out_dir.mkdir(parents=True, exist_ok=True)
                name = f264.stem
                out_path = out_dir / f"{name}.yuv"
                width, height = map(int, res.split('x'))
                cmd = [
                    "ffmpeg", "-y", "-s", f"{width}x{height}", "-i", str(f264),
                    "-pix_fmt", "yuv420p", str(out_path)
                ]
                run_command(cmd)


In [ ]:
# Example usage
input_pristine = "your_input_264_folder"
output_pristine_yuv = "pristine_yuv420p"
output_encoded_264 = "encoded"
output_encoded_yuv = "encoded_yuv420p"

resolutions = ["1280x720", "960x540"]
crf_values = [20, 28, 35]

# Step 1: Decode pristine .264 to yuv
decode_pristine_to_yuv(input_pristine, output_pristine_yuv, resolutions)

# Step 2: Encode to h264 with CRF levels
encode_yuv_to_h264(Path(output_pristine_yuv), Path(output_encoded_264), resolutions, crf_values)

# Step 3: Decode back to yuv
decode_encoded_to_yuv(Path(output_encoded_264), Path(output_encoded_yuv), resolutions, crf_values)
